## ScikitLearn

## 标准化，也称去均值和方差按比例缩放

数据集的 标准化 对scikit-learn中实现的大多数机器学习算法来说是 常见的要求 。如果个别特征或多或少看起来不是很像标准正态分布(具有零均值和单位方差)，那么它们的表现力可能会较差。

在实际情况中,我们经常忽略特征的分布形状，直接经过去均值来对某个特征进行中心化，再通过除以非常量特征(non-constant features)的标准差进行缩放。

例如，在机器学习算法的目标函数(例如SVM的RBF内核或线性模型的l1和l2正则化)，许多学习算法中目标函数的基础都是假设所有的特征都是零均值并且具有同一阶数上的方差。如果某个特征的方差比其他特征大几个数量级，那么它就会在学习算法中占据主导位置，导致学习器并不能像我们说期望的那样，从其他特征中学习。

In [2]:
from sklearn import preprocessing
import numpy as np

In [3]:
X_train = np.array(
[
    [ 1., -1.,  2.],
    [ 2.,  0.,  0.],
    [ 0.,  1., -1.]
])

In [4]:
X_train

array([[ 1., -1.,  2.],
       [ 2.,  0.,  0.],
       [ 0.,  1., -1.]])

In [5]:
np.mean(X_train)

0.4444444444444444

In [6]:
np.std(X_train)

1.0657403385139377

In [7]:
X_scale = preprocessing.scale(X_train)

In [8]:
X_scale

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

In [9]:
np.mean(X_scale)

4.9343245538895844e-17

In [10]:
np.std(X_scale)

1.0

In [13]:
X_scale.mean(axis=0)

array([0., 0., 0.])

In [14]:
X_scale.std(axis=0)

array([1., 1., 1.])

**标准化,将数据集的均值方差归零.**

**归一化，即将数据统一映射到[0,1]区间上**

预处理模块还提供实用程序类StandardScaler，其实现Transformer API以计算训练集上的均值和标准偏差，以便稍后能够在测试集上重新应用相同的变换。因此，此类适用于sklearn.pipeline.Pipeline的早期步骤：

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
standard_scaler = StandardScaler()

In [17]:
standard_scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [20]:
X_s_scaled = standard_scaler.fit_transform(X_train)

In [22]:
X_s_scaled == X_scale

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [23]:
standard_scaler.mean_

array([1.        , 0.        , 0.33333333])

In [24]:
standard_scaler.scale_

array([0.81649658, 0.81649658, 1.24721913])

### 将特征缩放至特定范围内

In [27]:
X_train

array([[ 1., -1.,  2.],
       [ 2.,  0.,  0.],
       [ 0.,  1., -1.]])

In [28]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

In [31]:
minmaxscaler = MinMaxScaler().fit(X_train)
minmaxscaler

MinMaxScaler(copy=True, feature_range=(0, 1))

In [32]:
X_minmax_scaled = minmaxscaler.transform(X_train)
X_minmax_scaled

array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

In [34]:
minmaxscaler.min_

array([0.        , 0.5       , 0.33333333])

In [35]:
minmaxscaler.scale_

array([0.5       , 0.5       , 0.33333333])

In [39]:
np.max(X_minmax_scaled)

1.0

In [40]:
np.min(X_minmax_scaled)

0.0

In [42]:
max_scaler = MaxAbsScaler().fit(X_train)

In [43]:
max_scaler.max_abs_

array([2., 1., 2.])

In [45]:
max_scaler.scale_

array([2., 1., 2.])

In [46]:
X_max_abs_scaled=max_scaler.transform(X_train)

In [47]:
np.max(X_max_abs_scaled)

1.0

In [48]:
np.min(X_max_abs_scaled)

-1.0

### 缩放稀疏数据

将稀疏数据居中会破坏数据中的稀疏结构，因此很少是明智之举。但是，缩放稀疏输入是有意义的，尤其是在特征不同的情况下。

MaxAbsScaler和maxabs_scale是专门为缩放稀疏数据设计的，都去这是推荐的方式。 但是，scale和StandardScaler可以接受scipy.sparse矩阵作为输入，只要将with_mean = False显式传递给构造函数即可。 否则会引发ValueError，因为静默居中会破坏稀疏性，并且通常会无意中分配过多的内存而导致执行崩溃。 RobustScaler无法适应稀疏输入，但您可以在稀疏输入上使用变换方法。

### 缩放具有离群值的数据

如果您的数据包含许多异常值，则使用数据的均值和方差进行缩放可能效果不佳。在这些情况下，您可以使用robust_scale和RobustScaler作为替代。他们对数据的中心和范围使用更可靠的估计。

### Scaling vs Whitening

由于下游模型可以进一步对特征的线性独立性做出一些假设，有时不足以独立地对中心和缩放特征。

要解决此问题，您可以使用带有whiten = True的sklearn.decomposition.PCA来进一步消除特征之间的线性相关性。

## 非线性变换


类似于缩放， QuantileTransformer 类将每个特征缩放在同样的范围或分布情况下。但是，通过执行一个秩转换能够使异常的分布平滑化，并且能够比缩放更少地受到离群值的影响。但是它的确使特征间及特征内的关联和距离失真了。

### Mapping to a Uniform distribution(均匀分布)

In [51]:
from sklearn.datasets import load_iris

In [52]:
from sklearn.model_selection import train_test_split

In [53]:
Iris = load_iris()

In [54]:
X = Iris.data
y = Iris.target

In [55]:
X_train,X_test,y_train,y_test =  train_test_split(X,y)

In [56]:
quantile_transformer  = preprocessing.QuantileTransformer().fit(X)

In [59]:
X_train_quantile = quantile_transformer.transform(X_train)

In [60]:
X_train[:10,:]

array([[6.8, 2.8, 4.8, 1.4],
       [6.4, 3.2, 4.5, 1.5],
       [5.1, 3.5, 1.4, 0.2],
       [5.2, 4.1, 1.5, 0.1],
       [7.7, 3. , 6.1, 2.3],
       [6.3, 2.5, 4.9, 1.5],
       [5. , 3.6, 1.4, 0.2],
       [6. , 2.2, 4. , 1. ],
       [4.4, 3. , 1.3, 0.2],
       [6.5, 2.8, 4.6, 1.5]])

In [61]:
X_train_quantile[:10,:]

array([[8.79379379e-01, 2.65265265e-01, 6.47647648e-01, 5.46546547e-01],
       [7.45245245e-01, 6.71171171e-01, 5.53553554e-01, 6.14114114e-01],
       [2.41741742e-01, 8.55855856e-01, 1.14114114e-01, 1.27627628e-01],
       [2.85285285e-01, 9.86577181e-01, 2.01201201e-01, 9.99999998e-08],
       [9.91491491e-01, 4.66466466e-01, 9.52952953e-01, 9.35935936e-01],
       [6.91191191e-01, 9.70970971e-02, 6.77677678e-01, 6.14114114e-01],
       [1.77677678e-01, 8.89389389e-01, 1.14114114e-01, 1.27627628e-01],
       [5.74074074e-01, 1.35135135e-02, 4.22422422e-01, 3.55855856e-01],
       [1.35135135e-02, 4.66466466e-01, 4.70470470e-02, 1.27627628e-01],
       [7.85285285e-01, 2.65265265e-01, 5.90590591e-01, 6.14114114e-01]])

In [62]:
np.percentile(X_train[:,0],[25,50,75,100])

array([5.1, 5.8, 6.4, 7.7])

In [64]:
np.percentile(X_train_quantile[:,0],[25,50,75,100])

array([0.24174174, 0.51001001, 0.74524525, 0.99149149])

## Mapping to a Gaussian distribution(高斯分布)

在许多建模方案中，期望数据集中的特征的正态性(normality)。PowerTransform是一系列参数单调变换，旨在将来自任何分布的数据映射到尽可能接近高斯分布，以便稳定方差并最小化偏度。

In [70]:
pt = preprocessing.PowerTransformer(method='box-cox').fit(X_train)

In [71]:
pt

PowerTransformer(copy=True, method='box-cox', standardize=True)

In [67]:
X_train_power = pt.transform(X_train)

In [69]:
X_train_power[:,0]

array([ 1.20363966,  0.76077617, -0.91027299, -0.76679927,  2.10566882,
        0.64546095, -1.05665889,  0.28755346, -2.00269044,  0.87419705,
        0.76077617,  1.09557649, -1.20606611,  0.76077617,  0.87419705,
        0.64546095,  0.28755346,  1.09557649,  0.03833559, -0.22003244,
        2.10566882,  0.64546095, -0.08966677, -0.35284187, -1.05665889,
        0.87419705, -0.48817953, -0.48817953,  0.03833559,  0.28755346,
       -0.08966677,  0.03833559, -0.08966677,  0.87419705, -0.48817953,
        1.81822098,  0.64546095,  0.28755346, -1.35860984, -0.91027299,
       -1.05665889, -0.91027299,  0.28755346, -0.35284187, -0.22003244,
       -1.20606611,  0.03833559, -0.08966677,  1.20363966, -1.05665889,
       -0.35284187,  0.40891187, -1.05665889,  1.09557649,  0.52819314,
       -1.35860984, -0.91027299,  0.16405129, -1.67360036,  0.76077617,
        1.6195053 ,  0.52819314,  0.03833559, -1.20606611, -0.76679927,
       -0.22003244, -0.48817953, -1.35860984,  0.52819314, -1.51

## 归一化

归一化 是 缩放单个样本以具有单位范数 的过程。如果你计划使用二次形式(如点积或任何其他核函数)来量化任何样本间的相似度，则此过程将非常有用。

In [72]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]

In [75]:
X_norm = preprocessing.normalize(X,norm='l2')

In [76]:
X_norm

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

In [77]:
normalizer = preprocessing.Normalizer().fit(X)
normalizer

Normalizer(copy=True, norm='l2')

In [78]:
normalizer.transform(X)

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

## 分类特征编码

In [79]:
enc = preprocessing.OrdinalEncoder()

In [80]:
enc

OrdinalEncoder(categories='auto', dtype=<class 'numpy.float64'>)

In [81]:
X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)
enc

OrdinalEncoder(categories='auto', dtype=<class 'numpy.float64'>)

In [83]:
enc.categories_

[array(['female', 'male'], dtype=object),
 array(['from Europe', 'from US'], dtype=object),
 array(['uses Firefox', 'uses Safari'], dtype=object)]

In [82]:
enc.transform([['female', 'from US', 'uses Safari']])

array([[0., 1., 1.]])

In [87]:
test_enc = [['female', 'from US', 'uses Safari']]

In [85]:
enc = preprocessing.OneHotEncoder()
enc.fit(X)

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values=None, sparse=True)

In [93]:
test_enc_trans = enc.transform(test_enc)

In [95]:
print(test_enc_trans)

  (0, 0)	1.0
  (0, 3)	1.0
  (0, 5)	1.0


In [96]:
test_enc_trans.toarray()

array([[1., 0., 0., 1., 0., 1.]])

In [97]:
enc = preprocessing.OneHotEncoder(handle_unknown='ignore')
enc.fit(X)

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)

In [100]:
test_enc2 = [['female', 'from Asia', 'uses Chrome']]

In [101]:
enc.transform(test_enc2).toarray()

array([[1., 0., 0., 0., 0., 0.]])

In [104]:
enc = preprocessing.OneHotEncoder(drop='first').fit(X)

TypeError: __init__() got an unexpected keyword argument 'drop'

## 离散化

In [105]:
X = np.array([[ -3., 5., 15 ],
              [  0., 6., 14 ],
              [  6., 3., 11 ]])

In [108]:
kbinsDiscretizer = preprocessing.KBinsDiscretizer(n_bins=[3,2,2],encode='ordinal')

In [109]:
kbinsDiscretizer.fit(X)

KBinsDiscretizer(encode='ordinal', n_bins=[3, 2, 2], strategy='quantile')

In [110]:
kbinsDiscretizer.transform(X)

array([[0., 1., 1.],
       [1., 1., 1.],
       [2., 0., 0.]])

**不同的离散化策略,uniform,quantile,kmeans**

KBinsDiscretizer implements different binning strategies, which can be selected with the strategy parameter. The ‘uniform’ strategy uses constant-width bins. The ‘quantile’ strategy uses the quantiles values to have equally populated bins in each feature. The ‘kmeans’ strategy defines bins based on a k-means clustering procedure performed on each feature independently.



### 特征二值化

In [111]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]

In [112]:
binarizer = preprocessing.Binarizer()
binarizer

Binarizer(copy=True, threshold=0.0)

In [113]:
binarizer.fit(X)

Binarizer(copy=True, threshold=0.0)

In [114]:
binarizer.transform(X)

array([[1., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

## 生成多项式特征

In [115]:
X = np.arange(6).reshape(3,2)

In [116]:
X

array([[0, 1],
       [2, 3],
       [4, 5]])

In [117]:
poly_gen = preprocessing.PolynomialFeatures()
poly_gen.fit(X)

PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)

In [118]:
poly_gen.transform(X)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

In [120]:
X = np.arange(9).reshape(3,3)

In [122]:
poly_gen = preprocessing.PolynomialFeatures(interaction_only=True)
poly_gen.fit(X)

PolynomialFeatures(degree=2, include_bias=True, interaction_only=True)

In [123]:
poly_gen.transform(X)

array([[ 1.,  0.,  1.,  2.,  0.,  0.,  2.],
       [ 1.,  3.,  4.,  5., 12., 15., 20.],
       [ 1.,  6.,  7.,  8., 42., 48., 56.]])

## 自定义transformer

In [125]:
from sklearn.preprocessing import FunctionTransformer
transformer = FunctionTransformer(np.log1p, validate=True)
X = np.array([[0, 1], [2, 3]])
transformer.transform(X)

array([[0.        , 0.69314718],
       [1.09861229, 1.38629436]])